In [3]:
import pandas as pd # import all the ness packages for the code to work. 
import yaml
from sqlalchemy import create_engine
import os

class RDSDatabaseConnector: 
    def __init__(self, config_path, query):
        self.config_path = config_path
        self.query = query
    """ 
    
    Here the RDS Database connector class is intilized and then given the parameters of
    of the config path, or where to save the extracted data as a .YAML file. 
    and the query, which will USE the SQL language through SQLAlchemy, to select all the necesary columns
    from the database that I desire. 

    """

    def initialize_engine(self):
        try: # try is used for error handling.
            with open(self.config_path, 'r') as f: # the file at the call path parameter I designated is opened on read mode.
                credentials = yaml.safe_load(f) # the .YAML file is loaded with safe_load for security.
            if not credentials: # if the file can't safe load, an error with a explainatory message is thrown.
                raise ValueError("YAML file is empty or improperly formatted.")  

            df_user = credentials['RDS_USER']
            df_password = credentials['RDS_PASSWORD']
            df_host = credentials['RDS_HOST']
            df_name = credentials['RDS_DATABASE']
            df_port = credentials.get('RDS_PORT', 5432) 
            """

            Here the credentials provided in the worksheet are used to access the AWS RDS database.
            Each credential is given an alias to use when creating the engine.
            Using .get for the port instead directly accessing like the others ensures the code doesn't fail. 
            Port number is optional, so if it doesn't exist returns the default port number rather than
            stopping execution of code. 

            """

            df_url = f"postgresql://{df_user}:{df_password}@{df_host}:{df_port}/{df_name}" # credental aliases inserted in db url format.
            engine = create_engine(df_url) # engine created 
            return engine
        
        

        except Exception as e:  # generic error handling if anything goes wrong error message is printed.
            raise RuntimeError(f"An error occurred while initializing the engine: {e}")
        

    def fetch_data(self, engine):
        try:
            df = pd.read_sql(self.query, engine)
            return df
        except Exception as e:
            raise RuntimeError(f"Failed to fetch data: {e}")
    """

    Here a function to use Pandas (pd) to read the SQL commands used retrieve desired columns from the RDS db
    and create a pd dataframe (df). 
    
    """

    def save_data_to_csv(self, df, file_path):
        try:
            os.makedirs(os.path.dirname(file_path), exist_ok=True)  # ensures directory exists
            df.to_csv(file_path, index=False) # saves the pd df as a .csv file, and resets the index. 
            print(f"Data saved successfully to {file_path}")
        except Exception as e: 
            raise RuntimeError(f"Failed to save data to CSV: {e}")
        
        """
        Maybe not completey ness, but function that checks the filepath is legit,
        then saves the df as a .csv file, within the RDSDatabaseConnector Class. 
        
        """


if __name__ == "__main__":
    config_path = "/Users/max/coding_resources/finance_loan_project/flp.gitignore/credentials.yaml" # .YAML credentials filepath.
    query = """ 
        SELECT
            id,
            member_id,
            loan_amount,
            funded_amount,
            funded_amount_inv,
            term,
            int_rate,
            instalment,
            grade,
            sub_grade,
            employment_length,
            home_ownership,
            annual_inc,
            verification_status,
            issue_date,
            loan_status,
            payment_plan,
            purpose,
            dti,
            delinq_2yrs,
            earliest_credit_line,
            inq_last_6mths,
            mths_since_last_delinq,
            mths_since_last_record,
            open_accounts,
            total_accounts,
            out_prncp,
            out_prncp_inv,
            total_payment,
            total_payment_inv,
            total_rec_prncp,
            total_rec_int,
            total_rec_late_fee,
            recoveries,
            collection_recovery_fee,
            last_payment_date,
            last_payment_amount,
            next_payment_date,
            last_credit_pull_date,
            collections_12_mths_ex_med,
            mths_since_last_major_derog,
            policy_code,
            application_type
        FROM
            loan_payments
    """ # SQL query accessing all of the columns in the RDS db, all column names are used rather than * for thorough practice. 


    connector = RDSDatabaseConnector(config_path, query) # Class activated, with credentials.yaml and extraction instructions. 

    try:
        engine = connector.initialize_engine() # engine connection to db is intialized with message of conformation.
        print("Database engine initialized successfully.")

        flp_df = connector.fetch_data(engine) # using the query the wanted db data is fetched and converted into a pd df
        print("Data fetched successfully:") # conformation message. 
        print(flp_df) # pd db printed for futher conformation. 


        csv_file_path = "/Users/max/coding_resources/finance_loan_project/flp_df/flp_df_og2.csv" # directory location 
        connector.save_data_to_csv(flp_df, csv_file_path) # pd df of the wanted RDS db data saved as a .csv. whoo!. 

    except Exception as e: # last generic error handling. 
        print(f"Error: {e}")

Database engine initialized successfully.
Data fetched successfully:
             id  member_id  loan_amount  funded_amount  funded_amount_inv  \
0      38676116   41461848         8000         8000.0             8000.0   
1      38656203   41440010        13200        13200.0            13200.0   
2      38656154   41439961        16000        16000.0            16000.0   
3      38656128   41439934        15000        15000.0            15000.0   
4      38656121   41439927        15000        15000.0            15000.0   
...         ...        ...          ...            ...                ...   
54226     76597      76583         5000         5000.0             1775.0   
54227    142608      74724         5000         5000.0             2350.0   
54228    117045      70978         3500         3500.0             2225.0   
54229     88854      70699         5000         5000.0              225.0   
54230     72323      70694         5000         5000.0              350.0   

      